# Different Geotab APIs Input and Outputs

# Table of Contents

1. [Housekeeping](#housekeeping)

2. [Create Custom Zone](#create-custom-zone)  
    2.1 [Create Custom Zone Type](#create-custom-zone-type)  
    2.2 [Prepare Geometry for Creating Custom Zone](#prepare-geometry-for-creating-custom-zone)   
    2.3 [Create a Custom Zone](#create-a-custom-zone)  

3. [Get Custom Zones](#get-custom-zones)  

4. [Get Vehicle Classes](#get-vehicle-classes)  

5. [Get Harsh Events](#get-harsh-events)  
    5.1 [Vehicle Type Filters](#vehicle-type-filters)  
    5.2 [Accessing Event Data](#accessing-event-data)  

6. [Get Geotab Basemap for the Above Data](#get-geotab-basemap-for-the-above-data)  

7. [Get Segment Level Speed Data](#get-segment-level-speed-data)  

8.  [Get Speed Data for the Same Segment](#get-speed-data-for-the-same-segment)  

9.  [Get Speed Data for the Zone (Combined Segments; No Observed Counts)](#get-speed-data-for-the-zone-combined-segments-no-observed-counts)


## Housekeeping

In [ ]:
from mygeotab.altitude import AltitudeAPI
from datetime import datetime, date, time
import json
import getpass
import os
import json
import time
import shutil
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import warnings
warnings.filterwarnings("ignore")

client = AltitudeAPI(
    username="a-bibeka@tti.tamu.edu",
    password=getpass.getpass("Password: "),
    database="fhwa_project",
    server="altitudeserver.geotab.com",
)
credentials = client.authenticate()

In [8]:
from utils import geotabAPI


## Create Custom Zone

### Create Custom Zone Type

In [ ]:
# crtzonetypes_query = {
# "customZoneTypeName": "testZoneType",
# "comments": "Test Zone Types",
# "queryType":"createCustomZoneType"
# }
# api_params = {
#     "functionParameters": crtzonetypes_query,
#     "serviceName": "dna-altitude-zones",
# }
# api_results = client.do(api_params)
# api_results

2025-04-02 13:54:22.559 INFO     creating job
2025-04-02 13:54:23.319 INFO     job created: {'id': 'its_17436200632820_ei7b9utwfi8', 'status': {'state': 'RUNNING'}}
2025-04-02 13:54:23.320 INFO     checking the job status
2025-04-02 13:54:28.891 INFO     waiting for results: {'id': 'its_17436200632820_ei7b9utwfi8', 'status': {'state': 'RUNNING'}}
2025-04-02 13:54:29.451 INFO     job finished: {'id': 'its_17436200632820_ei7b9utwfi8', 'status': {'state': 'DONE'}}
2025-04-02 13:54:29.452 INFO     gathering result
2025-04-02 13:54:30.434 INFO     total number of rows: 1
2025-04-02 13:54:30.435 INFO     data gathered


[{'customZoneTypeId': 'zt-grpv9bjz6z9'}]

#### Custom Zone Type Id:

[{'customZoneTypeId': 'zt-grpv9bjz6z9'}]

### Prepare Geometry for Creating Custom Zone

In [113]:
from pathlib import Path

#### Dummy File with Shapes
**This is specfic to the person doing the analysis. I am just using a dummy file to show the geopandas to json to Geotab geometry conversion I am using.**

In [ ]:
pa_wz_dir = r"C:\Users\A-Bibeka\OneDrive - Texas A&M Transportation Institute\Projects\wzCrashRiskCMV\Data"
pa_wz_dir = Path(pa_wz_dir)
pa_fi = pa_wz_dir.joinpath("HWY_COND_LN.geojson")
wz_gpd = gpd.read_file(str(pa_fi))

#### Create Buffer for Custom Zone

In [117]:
wz_gpd_sample = wz_gpd.sample(2)
wz_gpd_sample = wz_gpd_sample.to_crs("EPSG:3081")
wz_gpd_sample["geometry"] = wz_gpd_sample.geometry.buffer(100)
wz_gpd_sample = wz_gpd_sample.to_crs("4326")
wz_gpd_sample_json = json.loads(wz_gpd_sample.to_json())
str(wz_gpd_sample_json["features"][0]['geometry'])

"{'type': 'Polygon', 'coordinates': [[[-94.86605001599187, 33.578307848700746], [-94.86605280476128, 33.578308417264466], [-94.87687236951969, 33.5805265411112], [-94.88550057672845, 33.58228136936236], [-94.88621282013914, 33.58241654924394], [-94.88677459662172, 33.5825042956026], [-94.88823754854133, 33.582687849946815], [-94.89365614993336, 33.58327894152079], [-94.8936601665239, 33.583279386010815], [-94.89581935642033, 33.583521765737075], [-94.89640368963065, 33.58358403132426], [-94.89650790471173, 33.5835995179835], [-94.89660980439652, 33.58362347954306], [-94.89670840736068, 33.58365568524591], [-94.89680276402562, 33.583695824941266], [-94.89689196570258, 33.58374351207115], [-94.89697515334312, 33.58379828739276], [-94.89705152581179, 33.583859623400905], [-94.89712034760119, 33.583926929407646], [-94.89718095591526, 33.583999557230605], [-94.89723276705261, 33.584076807434876], [-94.8972752820282, 33.5841579360684], [-94.89730809137946, 33.58424216182647], [-94.8973308791

##### Following is the output that we will use to feed into the Geotab API:

"{'type': 'Polygon', 'coordinates': [[[-94.86605001599187, 33.578307848700746], [-94.86605280476128, 33.578308417264466], [-94.87687236951969, 33.5805265411112], [-94.88550057672845, 33.58228136936236], [-94.88621282013914, 33.58241654924394], [-94.88677459662172, 33.5825042956026], [-94.88823754854133, 33.582687849946815], [-94.89365614993336, 33.58327894152079], [-94.8936601665239, 33.583279386010815], [-94.89581935642033, 33.583521765737075], [-94.89640368963065, 33.58358403132426], [-94.89650790471173, 33.5835995179835], [-94.89660980439652, 33.58362347954306], [-94.89670840736068, 33.58365568524591], [-94.89680276402562, 33.583695824941266], [-94.89689196570258, 33.58374351207115], [-94.89697515334312, 33.58379828739276], [-94.89705152581179, 33.583859623400905], [-94.89712034760119, 33.583926929407646], [-94.89718095591526, 33.583999557230605], [-94.89723276705261, 33.584076807434876], [-94.8972752820282, 33.5841579360684], [-94.89730809137946, 33.58424216182647], [-94.89733087911038, 33.58432867357573], [-94.89734342573574, 33.584416638165486], [-94.89734561039586, 33.584505208451276], [-94.89733741202153, 33.584593531453336], [-94.89731890953826, 33.584680756571075], [-94.89729028110722, 33.58476604377498], [-94.8972518024105, 33.58484857169671], [-94.89720384399712, 33.58492754553953], [-94.8971468677153, 33.58500220473293], [-94.89708142226534, 33.585071830257746], [-94.89700813791569, 33.58513575157104], [-94.89692772043345, 33.5851933530642], [-94.89684094428758, 33.58524407999215], [-94.89674864518993, 33.58528744381618], [-94.89665171204682, 33.58532302690918], [-94.89655107839748, 33.585350486578186], [-94.89644771342296, 33.585369558364896], [-94.89634261261139, 33.58538005859279], [-94.89623678816994, 33.585381886136254], [-94.89613125927546, 33.585375023394604], [-94.89554334175585, 33.58531237593433], [-94.89553619850315, 33.585311594446864], [-94.89337540538122, 33.58506903507438], [-94.88794846100504, 33.584477033824456], [-94.88792799091576, 33.58447463322222], [-94.88643575542821, 33.58428740486058], [-94.88639780000626, 33.58428206123169], [-94.88579668543244, 33.58418817055779], [-94.88575603319325, 33.58418114068003], [-94.88501546321535, 33.58404058468512], [-94.88499935537752, 33.58403741853681], [-94.87636204189874, 33.582280739184554], [-94.87636021633507, 33.58228036641972], [-94.86554094026287, 33.58006230269974], [-94.86101667244256, 33.57914516200125], [-94.86091515342729, 33.579120107922364], [-94.86081705722404, 33.579086845605346], [-94.86072332857425, 33.57904569539246], [-94.8606348701529, 33.57899705359207], [-94.86055253387474, 33.57894138866184], [-94.86047711268937, 33.57887923669658], [-94.86040933294453, 33.57881119626508], [-94.86034984739084, 33.57873792264511], [-94.86029922889578, 33.578660121512236], [-94.86025796492704, 33.578578542143546], [-94.86022645285851, 33.57849397020098], [-94.86020499614422, 33.57840722016516], [-94.86019380139679, 33.57831912749076], [-94.86019297639875, 33.57823054056084], [-94.86020252906559, 33.578142312516086], [-94.86022236737071, 33.578055293038716], [-94.86025230023277, 33.577970320169676], [-94.86029203935699, 33.57788821223794], [-94.86034120201262, 33.57780975997982], [-94.86039931471947, 33.57773571892408], [-94.86046581780866, 33.577666802116106], [-94.86054007081282, 33.57760367325127], [-94.86062135863445, 33.57754694028367], [-94.86070889843278, 33.577497149571585], [-94.86080184716259, 33.5774547806162], [-94.86089930969293, 33.57742024144421], [-94.86100034742697, 33.57739386467851], [-94.86110398734058, 33.57737590433533], [-94.86120923135198, 33.57736653337805], [-94.8613150659328, 33.577365842051826], [-94.86142047186789, 33.577373837014306], [-94.86152443406966, 33.577390441271675], [-94.86605001599187, 33.578307848700746]]]}"

### Create a Custom Zone:
1. **Note there is another option of uploading a geoJSON to the [Altitude Portal](https://altitudeportal.geotab.com/traffic-analytics)**  
2. **I am using one zone in the above example. It's possible to dissolve multiple zones into one single zone and then pull the results. It might be a faster way to pull the data as each API call takes several minutes.**

In [118]:
crtzone_query = {
"customZone": {
    "ZoneDescription": "TestZoneAxB",
    "CustomZoneTypes": ['zt-grpv9bjz6z9'],
    "Comments": "TestZoneAxB",
    "CustomerZoneId": "TestZoneAxB",
    "Geography": str(wz_gpd_sample_json["features"][0]['geometry'])
},
"queryType": "createCustomZone"
}
api_params = {
    "functionParameters": crtzone_query,
    "serviceName": "dna-altitude-zones",
}
api_results = client.do(api_params)

2025-04-02 14:06:29.252 INFO     creating job
2025-04-02 14:06:30.058 INFO     job created: {'id': 'its_17436207899540_0u5knj17s9mi', 'status': {'state': 'RUNNING'}}
2025-04-02 14:06:30.059 INFO     checking the job status
2025-04-02 14:06:35.586 INFO     waiting for results: {'id': 'its_17436207899540_0u5knj17s9mi', 'status': {'state': 'RUNNING'}}
2025-04-02 14:06:41.112 INFO     waiting for results: {'id': 'its_17436207899540_0u5knj17s9mi', 'status': {'state': 'RUNNING'}}
2025-04-02 14:06:46.940 INFO     waiting for results: {'id': 'its_17436207899540_0u5knj17s9mi', 'status': {'state': 'RUNNING'}}
2025-04-02 14:06:47.491 INFO     job finished: {'id': 'its_17436207899540_0u5knj17s9mi', 'status': {'state': 'DONE'}}
2025-04-02 14:06:47.495 INFO     gathering result
2025-04-02 14:06:49.181 INFO     total number of rows: 1
2025-04-02 14:06:49.184 INFO     data gathered


## Get Custom Zones

In [110]:
zonestat_query = {
"zoneStatuses":[1],
"queryType":"getCustomZones"
}
api_params = {
    "functionParameters": zonestat_query,
    "serviceName": "dna-altitude-zones",
}
api_results = client.do(api_params)
api_results

2025-04-02 13:49:58.812 INFO     creating job
2025-04-02 13:49:59.343 INFO     job created: {'id': 'its_17436197993440_cynjqt5pbte', 'status': {'state': 'RUNNING'}}
2025-04-02 13:49:59.344 INFO     checking the job status
2025-04-02 13:50:04.833 INFO     waiting for results: {'id': 'its_17436197993440_cynjqt5pbte', 'status': {'state': 'RUNNING'}}
2025-04-02 13:50:05.394 INFO     job finished: {'id': 'its_17436197993440_cynjqt5pbte', 'status': {'state': 'DONE'}}
2025-04-02 13:50:05.395 INFO     gathering result
2025-04-02 13:50:06.478 INFO     total number of rows: 1
2025-04-02 13:50:06.479 INFO     data gathered


[{'ZoneId': 'z-9z7rq44fz48',
  'ISO_3166_2': 'US-TX',
  'ZoneDescription': 'i20_test_zone',
  'CustomZoneTypes': [{'CustomZoneTypeId': 'zt-l576a69d4o',
    'CustomZoneTypeName': 'Test',
    'CustomZoneTypeStatus': 'Active'}],
  'Counties': [{'CountyId': '48363',
    'ISO_3166_2': 'US-TX',
    'IntersectPercentage': 1}],
  'Database': 'fhwa_project',
  'Comments': '',
  'ZoneStatus': 'Active',
  'CustomerZoneId': None,
  'CreatedTimeStamp': {'value': datetime.datetime(2025, 4, 2, 14, 12, 46, 621836, tzinfo=datetime.timezone.utc)},
  'CreatedBy': 'a-bibeka@tti.tamu.edu',
  'ModifiedTimeStamp': {'value': datetime.datetime(2025, 4, 2, 14, 12, 46, 621836, tzinfo=datetime.timezone.utc)},
  'ModifiedBy': 'a-bibeka@tti.tamu.edu',
  'Geography': '{"type":"Polygon","coordinates":[[[-98.23432035976856,32.565468717271415],[-98.2367086689518,32.56426764861084],[-98.23862527022268,32.563306734182476],[-98.24188836679416,32.5616913654103],[-98.247813712794,32.55907437995256],[-98.25045833024694,32.55

## Get Vehicle Classes

In [109]:
api_params = {
    "functionParameters": {"queryType": "getVehicleClasses", "vehicleClassSchemeId": 2},
    "serviceName": "dna-altitude-general",
}

api_results = client.do(api_params)
api_results

2025-04-02 13:48:52.965 INFO     creating job
2025-04-02 13:48:53.847 INFO     job created: {'id': 'its_17436197338350_3w685jszx7p', 'status': {'state': 'RUNNING'}}
2025-04-02 13:48:53.849 INFO     checking the job status
2025-04-02 13:48:59.260 INFO     waiting for results: {'id': 'its_17436197338350_3w685jszx7p', 'status': {'state': 'RUNNING'}}
2025-04-02 13:49:04.786 INFO     waiting for results: {'id': 'its_17436197338350_3w685jszx7p', 'status': {'state': 'RUNNING'}}
2025-04-02 13:49:05.312 INFO     job finished: {'id': 'its_17436197338350_3w685jszx7p', 'status': {'state': 'DONE'}}
2025-04-02 13:49:05.313 INFO     gathering result
2025-04-02 13:49:06.330 INFO     total number of rows: 7
2025-04-02 13:49:06.331 INFO     data gathered


[{'SchemeId': 2,
  'CategoryIndex': 0,
  'VehicleClassIndex': 0,
  'VehicleClass': 'Light Duty Trucks (< 10,000 lbs)',
  'WeightClasses': ['Class 1',
   'Class 2',
   'Class A',
   'Class B',
   'Class C',
   'Class D',
   'Class E',
   'Class F',
   'Class G',
   'Class H'],
  'MinWeightKilograms': 0,
  'MaxWeightKilograms': 4536,
  'MinWeightPounds': 0,
  'MaxWeightPounds': 10000,
  'CategoryDescription': 'GVWR',
  'VehicleType': 'Truck',
  'WeightClassGrouping': 'FHA - Gross Vehicle Weight Rating (GVWR)'},
 {'SchemeId': 2,
  'CategoryIndex': 0,
  'VehicleClassIndex': 1,
  'VehicleClass': 'Medium Duty Trucks (10,001 - 26,000 lbs)',
  'WeightClasses': ['Class 3', 'Class 4', 'Class 5', 'Class 6'],
  'MinWeightKilograms': 4537,
  'MaxWeightKilograms': 11793,
  'MinWeightPounds': 10001,
  'MaxWeightPounds': 26000,
  'CategoryDescription': 'GVWR',
  'VehicleType': 'Truck',
  'WeightClassGrouping': 'FHA - Gross Vehicle Weight Rating (GVWR)'},
 {'SchemeId': 2,
  'CategoryIndex': 0,
  'Vehic

## Get Harsh Events

6 calls per month
3 calls
    1 call for all days of the week
    2 calls for weekday and weekend separate
24 hours 

In [128]:
6 * 3 * 24

432

In [27]:
params_harsh = {
    "queryType": "getHarshEventsPerSegment",
    "dateRange": {"DateFrom": "2022-09-09", "DateTo": "2022-09-30"},
    "timeRange": {"TimeFrom": "00:00:00", "TimeTo": "23:59:59.999"},
    # "daysOfWeek": (7,1),
    "vehicleClasses": [
        # {"VehicleType": "Truck", "WeightClass": "Class 3"},
        # {"VehicleType": "Truck", "WeightClass": "Class 4"},
        # {"VehicleType": "Truck", "WeightClass": "Class 5"},
        # {"VehicleType": "Truck", "WeightClass": "Class 6"},
        {"VehicleType": "Truck", "WeightClass": "Class 7"},
        {"VehicleType": "Truck", "WeightClass": "Class 8"},
        # {"VehicleType": "Bus", "WeightClass": "*"},
        # {"VehicleType": "MPV", "WeightClass": "*"},
    ],
    "zones": [{"ZoneId": "z-9z7rq44fz48", "ISO_3166_2": "", "ZoneType": "Custom"}],
    "magnitudeBins": [2.32, 3.54, 5.43, 10, 15, 20],
    "isMetric": False,
    "isDirectional": True,
}
api = geotabAPI(client_=client, service_name="dna-altitude-traffic")
job = api.create_job(params_harsh)

In [28]:
params_harsh["jobId"] = api.jobId
results = api.wait_for_job_to_complete(params_harsh)
results

{'id': 'its_17436054526840_kij566czlra', 'status': {'state': 'DONE'}}

In [29]:
results_iterator = api.fetch_data(params_harsh)
data = []
indices = []
for data_page in results_iterator:
    page = [] if data_page is None else data_page["data"][0]
    data.extend(page)
# data = next(results_iterator)["data"][0]

In [30]:
df = pd.DataFrame(data)

In [31]:
df

,SegmentId,RoadType,Direction,CardinalDirection,SegmentLength,ObservedHarshEvents,ObservedAccelerationEvents,ObservedBrakingEvents,ObservedCorneringEvents,ObservedCount,HarshEventsPerTenThousandTrips,AccelerationEventsPerTenThousandTrips,BrakingEventsPerTenThousandTrips,CorneringEventsPerTenThousandTrips,HarshEventsByVehicleClass,HarshEventsByVocation,HarshEventsByIndustry,HarshEventsByHour,HarshEventsByMagnitude
0,-527470241087376505,motorway,1,NE,0.8600,1,0,1,0,6045,1.65,0.00,1.65,0,"[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedHarshEvents':...","[{'Hour': 0, 'ObservedHarshEvents': None, 'Obs...","[{'Magnitude': '(0, 2.32]', 'LowerBound': 0, '..."
1,6637336668568124620,motorway,1,SW,0.9260,6,2,4,0,5543,10.82,3.61,7.22,0,"[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedHarshEvents':...","[{'Hour': 0, 'ObservedHarshEvents': None, 'Obs...","[{'Magnitude': '(0, 2.32]', 'LowerBound': 0, '..."
2,8460291558183656345,motorway,1,W,2.4398,4,1,3,0,5555,7.20,1.80,5.40,0,"[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedHarshEvents':...","[{'Hour': 0, 'ObservedHarshEvents': None, 'Obs...","[{'Magnitude': '(0, 2.32]', 'LowerBound': 0, '..."
3,8627545379398567069,motorway,1,E,2.7428,3,0,3,0,6050,4.96,0.00,4.96,0,"[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedHarshEvents':...","[{'Hour': 0, 'ObservedHarshEvents': None, 'Obs...","[{'Magnitude': '(0, 2.32]', 'LowerBound': 0, '..."


In [32]:
df_exploded = df[["SegmentId", "Direction", "HarshEventsByMagnitude"]].explode(
    "HarshEventsByMagnitude"
)
df_flat = pd.json_normalize(df_exploded["HarshEventsByMagnitude"])
df_result = pd.concat(
    [df_exploded[["SegmentId", "Direction"]].reset_index(drop=True), df_flat], axis=1
)
df_result

,SegmentId,Direction,Magnitude,LowerBound,UpperBound,ObservedHarshEvents,ObservedAccelerationEvents,ObservedBrakingEvents,ObservedCorneringEvents
0,-527470241087376505,1,"(0, 2.32]",0.00,2.32,1,0,1,0
1,6637336668568124620,1,"(0, 2.32]",0.00,2.32,3,1,2,0
2,6637336668568124620,1,"(2.32, 3.54]",2.32,3.54,1,0,1,0
3,6637336668568124620,1,"(3.54, 5.43]",3.54,5.43,2,1,1,0
4,8460291558183656345,1,"(0, 2.32]",0.00,2.32,2,0,2,0
5,8460291558183656345,1,"(2.32, 3.54]",2.32,3.54,2,1,1,0
6,8627545379398567069,1,"(0, 2.32]",0.00,2.32,1,0,1,0
7,8627545379398567069,1,"(2.32, 3.54]",2.32,3.54,2,0,2,0


## Get Geotab Basemap for the Above Data

In [ ]:
osm_params = {
    "queryType": "getRoadSegments",
    "zones": [{"ZoneId": "z-9z7rq44fz48", "ISO_3166_2": "", "ZoneType": "Custom"}],
    "asOfDate": "2022-09-01",
    "excludeServiceRoads": True,
    "roadTypes": [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
        "motorway_link",
        "trunk_link",
        "primary_link",
        "secondary_link",
        "tertiary_link",
    ],
    "resultsLimit": 50000,
}
api_osm = geotabAPI(client_=client, service_name="dna-altitude-general")
job = api_osm.create_job(osm_params)
osm_params["jobId"] = api_osm.jobId
results_osm = api_osm.wait_for_job_to_complete(osm_params)
results_iterator_osm = api_osm.fetch_data(osm_params)
data_osm = []
indices = []
for data_page in results_iterator_osm:
    page = [] if data_page is None else data_page["data"][0]
    data_osm.extend(page)
df_osm = pd.DataFrame(data_osm)
df_osm["Geography"] = df_osm.apply(lambda x: shape(eval(x["Geography"])), axis=1)


In [ ]:
df_osm

,AsOfDate,ReferenceDate,ISO_A3,Country,ISO_3166_2,StateId,State,CountyId,County,SegmentId,...,SegmentIndex,NodeFrom,NodeTo,RoadType,SegmentName,IsOneWay,CardinalDirectionStartingAtNodeFrom,CardinalDirectionStartingAtNodeTo,SpeedLimit,Geography
0,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,7284572957136258143,...,3,236191030,495350103,tertiary,I 20 Service Road,False,SW,NE,50,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
1,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,-5247455792902800851,...,0,236353451,236353447,motorway,I 20,True,NE,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
2,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,5359404166693966410,...,0,495350111,236108283,tertiary,I 20 Service Road,False,SW,NE,50,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
3,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,7057080509202709901,...,1,236110333,236191019,tertiary,I 20 Service Road,False,W,E,50,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
4,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,6388446693859103344,...,0,236352481,236352477,motorway,I 20,True,SW,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
5,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,4472000251645386229,...,2,236191019,236191030,tertiary,I 20 Service Road,False,SW,NE,50,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
6,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,-6146975812736024577,...,2,236191072,6591660945,tertiary,I 20 Service Road,False,W,E,50,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
7,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,8460291558183656345,...,2,236352601,236352481,motorway,I 20,True,W,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
8,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,6637336668568124620,...,0,236352477,236200714,motorway,I 20,True,SW,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
9,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,6732624970068961918,...,0,235939502,235939505,tertiary,I 20 Service Road,False,NE,SW,50,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."


In [126]:
gpdf_osm = gpd.GeoDataFrame(
    df_osm[[x for x in df_osm.columns if x != "Geography"]],
    geometry=gpd.GeoSeries(df_osm["Geography"].values),
)
gpdf_osm.crs = "4326"
gpdf_osm.head(2)

,AsOfDate,ReferenceDate,ISO_A3,Country,ISO_3166_2,StateId,State,CountyId,County,SegmentId,...,SegmentIndex,NodeFrom,NodeTo,RoadType,SegmentName,IsOneWay,CardinalDirectionStartingAtNodeFrom,CardinalDirectionStartingAtNodeTo,SpeedLimit,geometry
0,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,7284572957136258143,...,3,236191030,495350103,tertiary,I 20 Service Road,False,SW,NE,50,"LINESTRING (-98.24051 32.56343, -98.24364 32.5..."
1,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,-5247455792902800851,...,0,236353451,236353447,motorway,I 20,True,NE,None,75,"LINESTRING (-98.25056 32.55829, -98.24962 32.5..."


## Get Segment Level Speed Data
getSpeedMapMetrics

In [ ]:
params_mapmetrics = {
    "queryType": "getSpeedMapMetrics",
    "dateRange": {"DateFrom": "2022-09-01", "DateTo": "2022-09-30"},
    "timeRange": {"TimeFrom": "00:00:00", "TimeTo": "23:59:59.999"},
    "zones": [{"ZoneId": "z-9z7rq44fz48", "ISO_3166_2": "", "ZoneType": "Custom"}],
    "roadTypes":[
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
        "motorway_link",
        "trunk_link",
        "primary_link",
        "secondary_link",
        "tertiary_link",
    ],
    "isMetric": False,
}
api = geotabAPI(client_=client, service_name="dna-altitude-traffic")
job = api.create_job(params_mapmetrics)

In [25]:
params_mapmetrics["jobId"] = api.jobId
results = api.wait_for_job_to_complete(params_mapmetrics)
results_iterator = api.fetch_data(params_mapmetrics)
data = []
indices = []
for data_page in results_iterator:
    page = [] if data_page is None else data_page["data"][0]
    data.extend(page)
df_mapmetrics = pd.DataFrame(data)

In [26]:
df_mapmetrics

,SegmentId,RoadType,SegmentLength,SpeedLimit,ObservedCount,VehicleCount,SpotSpeedAvg,TravelSpeedAvg,TravelTimeAvg,SpotSpeedPercentiles
0,-2204090236541145661,tertiary,0.5504,50,NaN,NaN,NaN,NaN,NaN,[]
1,-5247455792902800851,motorway,0.0607,75,8070.0,3925.0,70.57,70.02,3.13,"[{'Percentile': 15, 'Value': 65.4}, {'Percenti..."
2,-527470241087376505,motorway,0.8600,75,8055.0,3920.0,71.21,70.42,44.16,"[{'Percentile': 15, 'Value': 66.5}, {'Percenti..."
3,-6146975812736024577,tertiary,0.8869,50,NaN,NaN,NaN,NaN,NaN,[]
4,-8263618753475850279,tertiary,0.0414,50,NaN,NaN,NaN,NaN,NaN,[]
5,-9085443231021542114,tertiary,0.2904,50,NaN,NaN,NaN,NaN,NaN,[]
6,2999538734240872823,tertiary,0.2110,50,NaN,NaN,NaN,NaN,NaN,[]
7,4472000251645386229,tertiary,0.3198,50,NaN,NaN,NaN,NaN,NaN,[]
8,5359404166693966410,tertiary,0.0528,50,NaN,NaN,NaN,NaN,NaN,[]
9,6388446693859103344,motorway,0.0543,75,7353.0,3559.0,70.55,70.07,2.80,"[{'Percentile': 15, 'Value': 66.14}, {'Percent..."


## Get Speed Data for the Same Segment
getSpeedAnalysisPerSegment

#### Input Paramter Notes:
1. `aggregationPeriod` can allow us to get the speed characteristics at defined intervals for the `timeRange`.
2. `isDirectional` is a weird parameter. It is useful for two-way undivided roadways, where you may want to get the data for the reverse direction. But, for freeways or interstates that are directional and have separate links for each direction, this parameter is not useful. It will return data for one direction and incorrectly use `Direction` column with `1` or `-1` to return `Null`.

In [93]:
params_speed = {
    "queryType": "getSpeedAnalysisPerSegment",
    "aggregationPeriod":{"Value": 15, "Unit":"minute"},
    "dateRange": {"DateFrom": "2022-09-01", "DateTo": "2022-09-30"},
    "timeRange": {"TimeFrom": "00:00:00", "TimeTo": "23:59:59.999"},
    "zones": [{"ZoneId": "z-9z7rq44fz48", "ISO_3166_2": "", "ZoneType": "Custom"}],
    "roadTypes":[
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
        "motorway_link",
        "trunk_link",
        "primary_link",
        "secondary_link",
        "tertiary_link",
    ],    "isMetric": False,
    "isDirectional": False,
}
api = geotabAPI(client_=client, service_name="dna-altitude-traffic")
job = api.create_job(params_speed)
params_speed["jobId"] = api.jobId
results = api.wait_for_job_to_complete(params_speed)
results_iterator = api.fetch_data(params_speed)
data = []
indices = []
for data_page in results_iterator:
    page = [] if data_page is None else data_page["data"][0]
    data.extend(page)
df_speed = pd.DataFrame(data)


In [104]:
test = df_speed.loc[lambda df: (df.RoadType.isin(["motorway", "motorway_link"])) & (df.AggregationPeriodFrom == time(0, 30))]

In [106]:
test

,SegmentId,RoadType,Direction,CardinalDirection,SegmentLength,AggregationPeriodFrom,ObservedCount,VehicleCount,DominantPace,SpeedLimit,...,TravelTimeHarmonic,TravelTimeDelayFromFreeFlow,TravelTimeReliability,SpotSpeedPercentiles,TravelSpeedPercentiles,TravelTimePercentiles,SampleSizePctByPace,ObservedCountByVehicleClass,ObservedCountByVocation,ObservedCountByIndustry
98,-5247455792902800851,motorway,0,,0.0607,00:30:00,68.0,62.0,"{'PaceFrom': 60, 'PaceTo': 70}",75.0,...,3.06,0.30,1.04,"[{'Percentile': 15, 'Value': 67.66}, {'Percent...","[{'Percentile': 15, 'Value': 67.4}, {'Percenti...","[{'Percentile': 15, 'Value': 2.89}, {'Percenti...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': 23, '..."
194,-527470241087376505,motorway,0,,0.8600,00:30:00,69.0,61.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,...,43.29,4.39,1.04,"[{'Percentile': 15, 'Value': 67.74}, {'Percent...","[{'Percentile': 15, 'Value': 67.67}, {'Percent...","[{'Percentile': 15, 'Value': 40.84}, {'Percent...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': 24, '..."
866,6388446693859103344,motorway,0,,0.0543,00:30:00,64.0,56.0,"{'PaceFrom': 60, 'PaceTo': 70}",75.0,...,2.80,0.32,1.07,"[{'Percentile': 15, 'Value': 65.01}, {'Percent...","[{'Percentile': 15, 'Value': 64.59}, {'Percent...","[{'Percentile': 15, 'Value': 2.6}, {'Percentil...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': 18, '..."
962,6637336668568124620,motorway,0,,0.9260,00:30:00,64.0,56.0,"{'PaceFrom': 60, 'PaceTo': 70}",75.0,...,47.55,5.18,1.09,"[{'Percentile': 15, 'Value': 65.23}, {'Percent...","[{'Percentile': 15, 'Value': 63.6}, {'Percenti...","[{'Percentile': 15, 'Value': 44.08}, {'Percent...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': 18, '..."
1346,8460291558183656345,motorway,0,,2.4398,00:30:00,65.0,56.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,...,124.00,13.27,1.09,"[{'Percentile': 15, 'Value': 68.99}, {'Percent...","[{'Percentile': 15, 'Value': 64.75}, {'Percent...","[{'Percentile': 15, 'Value': 116.63}, {'Percen...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': 19, '..."
1538,8627545379398567069,motorway,0,,2.7428,00:30:00,68.0,62.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,...,139.39,15.32,1.07,"[{'Percentile': 15, 'Value': 68.99}, {'Percent...","[{'Percentile': 15, 'Value': 65.52}, {'Percent...","[{'Percentile': 15, 'Value': 129.84}, {'Percen...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': 23, '..."


#### Output Notes:
1. The `ObservedCount` is defined as number traversals. `VehicleCount` as number of vehicles. Maybe when the same vehicle traverses the same segment multiple times, it is counted as `ObservedCount` but not `VehicleCount`.
2. The `segmentId`s are connected, if you use `NodeFrom` and `NodeTo`. These two form a linked-list that can be used to connect segments in order if you want the speed aggregates for the longer HPMS segments.
3. `TravelTimeAvg` is a good parameter as we can combine it across different segments. 
4. **Spot Speed**: from Geotab data dictionary: the maximum individual speed observed as a vehicle drives through selected road segments.
5. Cannot use the following in current form:
   1. *Percentiles* don't add. We can't use *Percentile95* from one segment and *Percentile95* from another segment to get the overall *Percentile95* for the combined segments.
   2. `TravelTimeStdev`: See following discussion on why we cannot add standard deviations or variances: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation. 
   3. `TravelSpeedPctOfFreeFlow`: The percentage of vehicle passes where the Travel Time speed was over the speed limit.. Different segments will have different ratios. We may be able to take the maximum or minimum of the ratios, but cannot use other aggregates.
   4. `TravelSpeedPctOverSpeedLimit`: The percentage of vehicle passes where the Travel Time speed was over the speed limit. Same issue as above. I don't like this measure as it depends on OSM speed limits. Can use it if we have other studies showing OSM speed limits are reliable across the U.S.
   5.  `SpotSpeedPctOfSpeedLimit`
   6.  `SpotSpeedPctOfFreeFlow`
   7.  `SpotSpeedStdev`
6. `getSpeedAnalysisSummary` in [Get Speed Data for the Zone (Combined Segments; No Observed Counts)](#get-speed-data-for-the-zone-combined-segments-no-observed-counts) has the ability to get around the above issues. But that would require us to set ordered segments in the API call using `segmentsDirectional` parameter (I think). This might be a risker approach. We would need clarification from Geotab on this.


In [89]:
df_osm.loc[lambda df: df.SegmentId.isin(test.SegmentId.unique()) & (df.CardinalDirectionStartingAtNodeFrom.isin(["E", "NE"]))]

,AsOfDate,ReferenceDate,ISO_A3,Country,ISO_3166_2,StateId,State,CountyId,County,SegmentId,...,SegmentIndex,NodeFrom,NodeTo,RoadType,SegmentName,IsOneWay,CardinalDirectionStartingAtNodeFrom,CardinalDirectionStartingAtNodeTo,SpeedLimit,Geography
1,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,-5247455792902800851,...,0,236353451,236353447,motorway,I 20,True,NE,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
11,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,-527470241087376505,...,3,236023440,236353451,motorway,I 20,True,NE,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."
16,{'value': 2022-09-01},{'value': 2022-08-21},USA,United States of America (the),US-TX,48,Texas,48363,Palo Pinto,8627545379398567069,...,0,236353447,236215782,motorway,I 20,True,E,None,75,"{ ""type"": ""LineString"", ""coordinates"": [ [-98...."


## Get Speed Data for the Zone (Combined Segments; No Observed Counts)

In [107]:
params_speed = {
    "queryType": "getSpeedAnalysisSummary",
    "aggregationPeriod":{"Value": 15, "Unit":"minute"},
    "dateRange": {"DateFrom": "2022-09-01", "DateTo": "2022-09-30"},
    "timeRange": {"TimeFrom": "00:00:00", "TimeTo": "23:59:59.999"},
    "zones": [{"ZoneId": "z-9z7rq44fz48", "ISO_3166_2": "", "ZoneType": "Custom"}],
    "roadTypes":[
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
        "motorway_link",
        "trunk_link",
        "primary_link",
        "secondary_link",
        "tertiary_link",
    ],    "isMetric": False,
    "isDirectional": True,
}
api = geotabAPI(client_=client, service_name="dna-altitude-traffic")
job = api.create_job(params_speed)
params_speed["jobId"] = api.jobId
results = api.wait_for_job_to_complete(params_speed)
results_iterator = api.fetch_data(params_speed)
data = []
indices = []
for data_page in results_iterator:
    page = [] if data_page is None else data_page["data"][0]
    data.extend(page)
df_speed_anyl = pd.DataFrame(data)


In [108]:
df_speed_anyl

,SegmentsDirectional,CorridorDirection,CardinalDirection,CorridorLength,AggregationPeriodFrom,ObservedCount,VehicleCount,DominantPace,SpeedLimitAvg,SpeedLimitMin,...,TravelTimeHarmonic,TravelTimeDelayFromFreeFlow,TravelTimeReliability,SpotSpeedPercentiles,TravelSpeedPercentiles,TravelTimePercentiles,SampleSizePctByPace,ObservedCountByVehicleClass,ObservedCountByVocation,ObservedCountByIndustry
0,[],1,W,None,00:00:00,None,149.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,75.0,...,363.51,41.19,1.09,"[{'Percentile': 15, 'Value': 65.26}, {'Percent...","[{'Percentile': 15, 'Value': 64.95}, {'Percent...","[{'Percentile': 15, 'Value': 336.36}, {'Percen...","[{'PaceFrom': 50, 'PaceTo': 60, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': 48, 'ObservedCount': None, '..."
1,[],1,W,None,00:15:00,None,117.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,75.0,...,359.28,31.44,1.07,"[{'Percentile': 15, 'Value': 66.5}, {'Percenti...","[{'Percentile': 15, 'Value': 65.06}, {'Percent...","[{'Percentile': 15, 'Value': 333.42}, {'Percen...","[{'PaceFrom': 50, 'PaceTo': 60, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': None,..."
2,[],1,W,None,00:30:00,None,123.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,75.0,...,360.10,38.79,1.08,"[{'Percentile': 15, 'Value': 66.75}, {'Percent...","[{'Percentile': 15, 'Value': 65.27}, {'Percent...","[{'Percentile': 15, 'Value': 336.87}, {'Percen...","[{'PaceFrom': 60, 'PaceTo': 70, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': None,..."
3,[],1,W,None,00:45:00,None,124.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,75.0,...,358.29,34.91,1.08,"[{'Percentile': 15, 'Value': 66.48}, {'Percent...","[{'Percentile': 15, 'Value': 65.07}, {'Percent...","[{'Percentile': 15, 'Value': 333.59}, {'Percen...","[{'PaceFrom': 50, 'PaceTo': 60, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': None, 'ObservedCount': None,..."
4,[],1,W,None,01:00:00,None,122.0,"{'PaceFrom': 70, 'PaceTo': 80}",75.0,75.0,...,363.31,39.06,1.07,"[{'Percentile': 15, 'Value': 67.74}, {'Percent...","[{'Percentile': 15, 'Value': 66.31}, {'Percent...","[{'Percentile': 15, 'Value': 340.65}, {'Percen...","[{'PaceFrom': 50, 'PaceTo': 60, 'SampleSizePct...","[{'SchemeId': 2, 'CategoryIndex': 0, 'VehicleC...","[{'VocationId': 5, 'Vocation': 'LongHaul', 'Ob...","[{'NAICS_Code_1': 48, 'ObservedCount': None, '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,[],-1,E,None,22:45:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
188,[],-1,E,None,23:00:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
189,[],-1,E,None,23:15:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
190,[],-1,E,None,23:30:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]


#### Observation

*The `getSpeedAnalysisSummary` query does not provide observed counts. It does not contain segment information. 

Also, it the zone contains multiple road types and road type filter is too loose than the averaging across segments will happen across segments of different road types.*

#### Try `segmentsDirectional` Parameter:

In [120]:
params_speed = {
    "queryType": "getSpeedAnalysisSummary",
    "aggregationPeriod":{"Value": 15, "Unit":"minute"},
    "dateRange": {"DateFrom": "2022-09-01", "DateTo": "2022-09-30"},
    "timeRange": {"TimeFrom": "00:00:00", "TimeTo": "23:59:59.999"},
    "segmentsDirectional":[
        {"SegmentId":"-5413282495872703901", "Direction":1},
        {"SegmentId":"-4350021053557322330", "Direction":1},
        {"SegmentId":"-5326185399566610743", "Direction":1}
    ],
    "roadTypes":[
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
        "motorway_link",
        "trunk_link",
        "primary_link",
        "secondary_link",
        "tertiary_link",
    ],    "isMetric": False,
    "isDirectional": True,
}
api = geotabAPI(client_=client, service_name="dna-altitude-traffic")
job = api.create_job(params_speed)
params_speed["jobId"] = api.jobId
results = api.wait_for_job_to_complete(params_speed)
results_iterator = api.fetch_data(params_speed)
data = []
indices = []
for data_page in results_iterator:
    page = [] if data_page is None else data_page["data"][0]
    data.extend(page)
df_speed_anyl = pd.DataFrame(data)


In [127]:
df_speed_anyl

,SegmentsDirectional,CorridorDirection,CardinalDirection,CorridorLength,AggregationPeriodFrom,ObservedCount,VehicleCount,DominantPace,SpeedLimitAvg,SpeedLimitMin,...,TravelTimeHarmonic,TravelTimeDelayFromFreeFlow,TravelTimeReliability,SpotSpeedPercentiles,TravelSpeedPercentiles,TravelTimePercentiles,SampleSizePctByPace,ObservedCountByVehicleClass,ObservedCountByVocation,ObservedCountByIndustry
0,"[{'SegmentId': '-5413282495872703901', 'Direct...",1,N,0.1953,00:00:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
1,"[{'SegmentId': '-5413282495872703901', 'Direct...",1,N,NaN,00:15:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
2,"[{'SegmentId': '-5413282495872703901', 'Direct...",1,N,NaN,00:30:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
3,"[{'SegmentId': '-5413282495872703901', 'Direct...",1,N,NaN,00:45:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
4,"[{'SegmentId': '-5413282495872703901', 'Direct...",1,N,0.1953,01:00:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,"[{'SegmentId': '-5326185399566610743', 'Direct...",-1,S,NaN,22:45:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
188,"[{'SegmentId': '-5326185399566610743', 'Direct...",-1,S,NaN,23:00:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
189,"[{'SegmentId': '-5326185399566610743', 'Direct...",-1,S,NaN,23:15:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
190,"[{'SegmentId': '-5326185399566610743', 'Direct...",-1,S,NaN,23:30:00,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
